In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator
import numpy as np
from SimPEG import PF, Utils, Mesh, Maps

import matplotlib.pyplot as plt
import ipywidgets as widgets

# Load data and topo and build default model
survey = Mag.readMagneticsObservations('./DIGHEM_Mag_floor10nt_25m.obs')
topo = np.genfromtxt('TKCtopoDwnS.dat', skip_header=1)
locs = survey.srcField.rxList[0].locs

# Build the problem
survey = ProblemSetter.setSyntheticProblem(locs, topo=topo, discretize=True, plotSections=True)


C:\Users\DominiqueFournier\Documents\GIT\GeoToolkit\library\Mag\Simulator.py:694: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(


# 2.1.1.	Magnetic Data Preparation, Gridding, and Visualization#


## Magnetic response of a simple geologic model


<img src="./images/SyntheticModel.png">

The magnetic response (total field anomaly) of the geologic model is calculated on a series of east-west flight lines running roughly perpendicular to the general strike of geologic boundaries and structures featured in the model. Flight line spacing is 200 m. Data was collected at a flight height of 40 m. Topography is assumed to be flat in this example. The magnetic data are in nanoteslas (NT). 

The magnetic field values applied here reflect the inclination, declination, and field strength for central British Columbia, Canada.  

An East-West profile through the magnetic data is shown. The profile location can be changed using the slider bars, and the number of sampling points can be varied.  



In [2]:
param = Simulator.ViewMagSurveyWidget(survey)
display(param)

interactive(children=(FloatSlider(value=-14.629972527676728, continuous_update=False, description='East', max=…

## Magnetic field effect on response

As discussed in Section 1, the magnetic response will depend on the inclination, declination, and field strength of the magnetic field at the survey location. Use the drop down menu to explore the range of responses that would be measured if this same geologic (and magnetic rock properties) scenario existed at different locations on Earth (sites represent current Mineral Deposit Research Unit project sites). Notice how data collected at very low latitides near the magnetic equator (e.g. Chile and Peru locations) show magnetic anomlies offset significantly from their source due to the near-horizontal magnetic inclinations existing at these locations. 


In [3]:
import json
# cities = shapefile.Reader('./assets/worldShapefile/ne_10m_populated_places_simple.shp')
worldFile = './assets/Continents/continent.shp'

# Load in the pre-defined locations
with open('./assets/MAGlocations_w_field.json', 'r') as fp:
    sites = json.load(fp)

view = Simulator.worldViewerWidget(worldFile, sites, locs)
display(view)

interactive(children=(Dropdown(description='Location:', options=('Yukon', 'Northern BC', 'Nevada', 'Colombia',…

## Gridding the magnetic data

Magnetic data should be gridded prior to applying transforms and filters. This means that the flight line data must be interpolated on a regular 2D grid. There are numerous possible ways to grid data. Several researchers have demonstrated that minimum curvature methods are robust and yield a sensible and smooth result, so we have chosen minimum curvature to complete the gridding for this example. In this example, the data are interpolated on a 10 m x 10 m grid. The magnetic field parameters of central BC (Geoscience BC Search II area) are returned to here. 

The resulting gridded data shows the larger and smaller magnetic bodies, and the magnetic low related to the NW trending structure cross-cutting the larger magnetic domain.

## Basic visualization

Simple enhancements to the image can help the user better visualize features within the magnetic data. Applying sun-shading from various angles, experimenting with colour contrast, and adding contour lines are a simple means to explore the data initially. 

Try using the slider bars below to change the direction (SunAzimuth) and angle (SunAngle) of sun shading, to vary color saturation and transparency, and to manipulate magnetic contours lines (MagContour).
   

-don’t think we need vScale


In [4]:
view = Simulator.dataHillsideWidget(survey)
display(view)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

## Advanced visualization

Addional image processing can be done to explore the range of magnetic responses within the data. Applying various color scales and stretches benefits the geophysical data interpretion as this provides a more complete overview of features in the data, which might be obscured when only one set of display parameters is applied.  